Example in 21.5.1 Murphy book
# Dataset

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
from __future__ import division
import numpy as np

mu_true, tau_true = 0, 1
N = 1000
data = np.random.normal(mu_true, tau_true, N)
data[:10]

array([-1.62552805,  1.81198748, -1.67392204, -1.01551505,  0.23891435,
       -1.10653698,  0.35336531,  0.64349335, -0.40868741, -0.22359277])

# Algorithm
Here we set the hyperparameters $\mu_0$, $\kappa_0$, $a_0$, $b_0$

In [4]:
a_0, b_0, mu_0, kappa_0 = 0, 0, 0, 0
m = np.mean(data)
truePost = dict()
truePost['mu'] = np.mean(data)
truePost['kappa'] = N
truePost['alpha'] = N/2
truePost['beta'] = 1 / 2 * np.sum((data - m)**2)

m
truePost

-0.024067928932618343

{'alpha': 500.0,
 'beta': 504.14338288861779,
 'kappa': 1000,
 'mu': -0.024067928932618343}

Iterative main

In [5]:
max_iter = 1000
iter = 0

while iter < max_iter:
    iter += 1
    
    a_N = a_0 + (N+1) / 2
    
    
iter

NameError: name 'a0' is not defined